In [ ]:
!pip install opencv-python --upgrade
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 as cv
import time
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import pickle
from keras.models import load_model

     |████████████████████████████████| 49.5MB 96kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [ ]:
print( cv.__version__ )

4.4.0


In [ ]:
!pip install kaggle -q
!mkdir .kaggle
import json
token = {"username":"someonetest","key":"8790cbd05d258df3a0fec2ffb25d689d"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json --force
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets download -d mlg-ulb/creditcardfraud -p /content --force
!kaggle datasets download -d valentynsichkar/traffic-signs-preprocessed -p /content --force
!unzip \*.zip

mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}
100% 4.16G/4.16G [00:53<00:00, 75.7MB/s]
100% 4.16G/4.16G [00:53<00:00, 83.0MB/s]
Archive:  traffic-signs-preprocessed.zip
  inflating: data0.pickle            
  inflating: data1.pickle            
  inflating: data2.pickle            
  inflating: data3.pickle            
  inflating: data4.pickle            
  inflating: data5.pickle            
  inflating: data6.pickle            
  inflating: data7.pickle            
  inflating: data8.pickle            
  inflating: datasets_preparing.py   
  inflating: label_names.csv         
  inflating: labels.pickle           
  inflating: mean_image_gray.pickle  
  inflating: mean_image_rgb.pickle   
  inflating: std_gray.pickle         
  inflating: std_rgb.pickle          
  inflating: test.pickle             
  inflating: train.pickle            
  inflating: valid.pickle            


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ln -s /content/drive/MyDrive/Projeto_Machine/Modelos/ /mydrive
!ls /mydrive

 Classificação	       'Resultado 5 épocas'   test_video.mp4
 localizacao	        test_3.jpg	      traffic-sign-to-test.mp4
 ResNet50_model2.h5     test_8.jpg	      yolov4-obj_best.weights
 ResNet50_model.h5      teste_image.png       yolov4-obj.cfg
'Resultado 10 épocas'   test_video2.mp4       yolov4-obj_last.weights


In [ ]:
labels = pd.read_csv('label_names.csv')
# labels

In [ ]:
# Loading trained CNN model to use it later when classifying from 4 groups into one of 43 classes
# model = load_model('/mydrive/ResNet50_model.h5')
model = load_model('/mydrive/ResNet50_model2.h5')

# Loading mean image to use for preprocessing further
# Opening file for reading in binary mode
with open('mean_image_rgb.pickle', 'rb') as f:
    mean = pickle.load(f, encoding='latin1')  # dictionary type
    
print(mean['mean_image_rgb'].shape)  # (3, 32, 32)

(3, 32, 32)


In [ ]:
# Trained weights can be found in the course mentioned above
path_to_weights = '/mydrive/yolov4-obj_last.weights'
path_to_cfg = '/mydrive/yolov4-obj.cfg'

#Using YOLOv3 to video
# path_to_weights = '/mydrive/localizacao/yolov3/backup/yolov3_custom_last.weights'
# path_to_cfg = '/mydrive/localizacao/yolov3/yolov3_custom.cfg'

# Loading trained YOLO v3 weights and cfg configuration file by 'dnn' library from OpenCV
network = cv.dnn_DetectionModel(path_to_cfg, path_to_weights)

# To use with GPU
network.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
network.setPreferableTarget(cv.dnn.DNN_TARGET_OPENCL_FP16)


In [ ]:
# Getting names of all YOLO v3 layers
layers_all = network.getLayerNames()

# Check point
# print(layers_all)

# Getting only detection YOLO v3 layers that are 82, 94 and 106
layers_names_output = [layers_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

# Check point
print()
print(layers_names_output)  # ['yolo_82', 'yolo_94', 'yolo_106']



['yolo_82', 'yolo_94', 'yolo_106']


In [ ]:
# Minimum probability to eliminate weak detections
probability_minimum = 0.3

# Setting threshold to filtering weak bounding boxes by non-maximum suppression
threshold = 0.3

# Generating colours for bounding boxes
# randint(low, high=None, size=None, dtype='l')
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

# Check point
print(type(colours))  # <class 'numpy.ndarray'>
print(colours.shape)  # (43, 3)
print(colours[0])  # [25  65 200]

<class 'numpy.ndarray'>
(43, 3)
[ 12  63 211]


In [ ]:
# Reading video from a file by VideoCapture object
# video = cv2.VideoCapture('../input/traffic-signs-dataset-in-yolo-format/traffic-sign-to-test.mp4')
# video = cv2.VideoCapture('../input/videofortesting/ts_video_1.mp4')
# video = cv.VideoCapture('/mydrive/traffic-sign-to-test.mp4')
video = cv.VideoCapture('/mydrive/test_video.mp4')
# video = cv.VideoCapture('/mydrive/test_video2.mp4')


# Writer that will be used to write processed frames
writer = None

# Variables for spatial dimensions of the frames
h, w = None, None

In [ ]:
%matplotlib inline

# Setting default size of plots
plt.rcParams['figure.figsize'] = (3, 3)

# Variable for counting total amount of frames
f = 0

# Variable for counting total processing time
t = 0

# Catching frames in the loop
while True:
    # Capturing frames one-by-one
    ret, frame = video.read()

    # If the frame was not retrieved
    if not ret:
        break
       
    # Getting spatial dimensions of the frame for the first time
    if w is None or h is None:
        # Slicing two elements from tuple
        h, w = frame.shape[:2]

    # Blob from current frame
    blob = cv.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    # Forward pass with blob through output layers
    network.setInput(blob)
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    # Increasing counters
    f += 1
    t += end - start

    # Spent time for current frame
    print('Frame number {0} took {1:.5f} seconds'.format(f, end - start))

    # Lists for detected bounding boxes, confidences and class's number
    bounding_boxes = []
    confidences = []
    class_numbers = []

    # Going through all output layers after feed forward pass
    for result in output_from_network:
        # Going through all detections from current output layer
        for detected_objects in result:
            # Getting 80 classes' probabilities for current detected object
            scores = detected_objects[5:]
            # Getting index of the class with the maximum value of probability
            class_current = np.argmax(scores)
            # Getting value of probability for defined class
            confidence_current = scores[class_current]

            # Eliminating weak predictions by minimum probability
            if confidence_current > probability_minimum:
                # Scaling bounding box coordinates to the initial frame size
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                # Getting top left corner coordinates
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                # Adding results into prepared lists
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)
                

    # Implementing non-maximum suppression of given bounding boxes
    results = cv.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

    # Checking if there is any detected object been left
    if len(results) > 0:
        # Going through indexes of results
        for i in results.flatten():
            # Bounding box coordinates, its width and height
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
            
            
            # Cut fragment with Traffic Sign
            c_ts = frame[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]
            # print(c_ts.shape)
            
            if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                pass
            else:
                # Getting preprocessed blob with Traffic Sign of needed shape
                blob_ts = cv.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
                blob_ts[0] = blob_ts[0, :, :, :] - mean['mean_image_rgb']
                blob_ts = blob_ts.transpose(0, 2, 3, 1)
                # plt.imshow(blob_ts[0, :, :, :])L
                # plt.show()

                # Feeding to the Keras CNN model to get predicted label among 43 classes
                scores = model.predict(blob_ts)

                # Scores is given for image with 43 numbers of predictions for each class
                # Getting only one class with maximum value
                prediction = np.argmax(scores)
                # print(labels['SignName'][prediction])


                # Colour for current bounding box
                colour_box_current = colours[class_numbers[i]].tolist()

                # Drawing bounding box on the original current frame
                cv.rectangle(frame, (x_min, y_min),
                              (x_min + box_width, y_min + box_height),
                              colour_box_current, 2)

                # Preparing text with label and confidence for current bounding box
                text_box_current = '{}: {:.4f}'.format(labels['SignName'][prediction],
                                                       confidences[i])

                # Putting text with label and confidence on the original image
                cv.putText(frame, text_box_current, (x_min, y_min - 5),
                            cv.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)


    # Initializing writer only once
    if writer is None:
        fourcc = cv.VideoWriter_fourcc(*'mp4v')

        # Writing current processed frame into the video file
        writer = cv.VideoWriter('result.mp4', fourcc, 25,
                                 (frame.shape[1], frame.shape[0]), True)

    # Write processed current frame to the file
    writer.write(frame)


# Releasing video reader and writer
video.release()
writer.release()


Frame number 1 took 1.27610 seconds
Frame number 2 took 1.25930 seconds
Frame number 3 took 1.27841 seconds
Frame number 4 took 1.25647 seconds
Frame number 5 took 1.25561 seconds
Frame number 6 took 1.25566 seconds
Frame number 7 took 1.28225 seconds
Frame number 8 took 1.25034 seconds
Frame number 9 took 1.27599 seconds
Frame number 10 took 1.28280 seconds
Frame number 11 took 1.24825 seconds
Frame number 12 took 1.25992 seconds
Frame number 13 took 1.26258 seconds
Frame number 14 took 1.25636 seconds
Frame number 15 took 1.28677 seconds
Frame number 16 took 1.27908 seconds
Frame number 17 took 1.26213 seconds
Frame number 18 took 1.25509 seconds
Frame number 19 took 1.26486 seconds
Frame number 20 took 1.25339 seconds
Frame number 21 took 1.25425 seconds
Frame number 22 took 1.25775 seconds
Frame number 23 took 1.26242 seconds
Frame number 24 took 1.26104 seconds
Frame number 25 took 1.27007 seconds
Frame number 26 took 1.25950 seconds
Frame number 27 took 1.25220 seconds
Frame numb

In [ ]:
print('Total number of frames', f)
print('Total amount of time {:.5f} seconds'.format(t))
print('FPS:', round((f / t), 1))

Total number of frames 989
Total amount of time 1256.34071 seconds
FPS: 0.8


In [ ]:
# Saving locally without committing
from IPython.display import FileLink

FileLink('result.mp4')


/content/result.mp4